# 🏋️‍♂️ Sistema Completo de Geração de Plano de Treino com LangGraph

Este notebook implementa um agente de IA para gerar planos de treino de musculação personalizados, utilizando LangGraph, RAG, busca web e cálculos matemáticos. O fluxo é minimalista, didático e interativo.

---


## 1️⃣ Importação de Bibliotecas e Verificação de Dependências

Esta célula garante que todas as bibliotecas necessárias estejam instaladas e prontas para uso.

In [59]:
import sys
import importlib
from pathlib import Path
import numpy as np
import json
import hashlib
from typing import Dict, List, Any, Tuple

# Função para verificar instalação de pacotes

def verificar_instalacao(pacote):
    try:
        importlib.import_module(pacote)
        return True
    except ImportError:
        return False

# Pacotes essenciais
pacotes = {
    'sentence_transformers': 'sentence_transformers',
    'numpy': 'numpy',
    'scikit_learn': 'sklearn',
    'langgraph': 'langgraph',
    'langchain_core': 'langchain_core',
    'reportlab': 'reportlab'
}

print("🔎 Verificando dependências...")
todos_ok = True
for nome_display, nome_modulo in pacotes.items():
    status = verificar_instalacao(nome_modulo)
    emoji = "✅" if status else "❌"
    print(f"{emoji} {nome_display}")
    if not status:
        todos_ok = False

if not todos_ok:
    print("\n⚠️ Execute: pip install sentence-transformers scikit-learn langgraph langchain-core reportlab")
else:
    print("\n🎉 Todas as dependências estão instaladas!")

🔎 Verificando dependências...
✅ sentence_transformers
✅ numpy
✅ scikit_learn
✅ langgraph
✅ langchain_core
✅ reportlab

🎉 Todas as dependências estão instaladas!


## 2️⃣ Carregamento da Base de Conhecimento e Chunks

Aqui carregamos a base de conhecimento fitness e os chunks para o mecanismo RAG.

In [60]:
# Função para carregar base de conhecimento fitness
def carregar_base_conhecimento():
    arquivo_base = Path("base_conhecimento_fitness.txt")
    if not arquivo_base.exists():
        print("⚠️ Arquivo base_conhecimento_fitness.txt não encontrado! Usando base demo.")
        return """
### EXERCÍCIOS BÁSICOS

**Supino reto**
Exercício fundamental para peitoral maior, deltóide anterior e tríceps.

**Agachamento**
Rei dos exercícios para quadríceps, glúteos e core.

### HIPERTROFIA
Séries: 3-4
Repetições: 8-12
Descanso: 60-90 segundos
"""
    try:
        with open(arquivo_base, 'r', encoding='utf-8') as f:
            conteudo = f.read()
        print(f"✅ Base carregada: {arquivo_base.stat().st_size} bytes")
        return conteudo
    except Exception as e:
        print(f"❌ Erro ao carregar: {e}")
        return ""

# Função para criar chunks da base
def criar_chunks(conteudo):
    secoes = conteudo.split('### ')
    chunks = []
    for i, secao in enumerate(secoes[1:], 1):
        linhas = secao.strip().split('\n')
        titulo = linhas[0] if linhas else f"Seção {i}"
        texto_secao = '### ' + secao.strip()
        chunks.append({
            'id': len(chunks),
            'texto': texto_secao,
            'secao': titulo,
            'tokens': len(texto_secao.split()),
            'hash': hashlib.md5(texto_secao.encode()).hexdigest()[:8]
        })
    print(f"📄 {len(chunks)} chunks criados")
    return chunks

# Carregar base e chunks
conhecimento_bruto = carregar_base_conhecimento()
chunks = criar_chunks(conhecimento_bruto)
print(f"\n📊 Total de chunks disponíveis: {len(chunks)}")

✅ Base carregada: 8822 bytes
📄 26 chunks criados

📊 Total de chunks disponíveis: 26


## 3️⃣ Definição dos Nós do LangGraph

Aqui estão as funções dos nós: coleta/validação, cálculo, busca web, RAG e geração do plano.

In [61]:
from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage
import random

# ===== DEFINIÇÃO DO ESTADO DO LANGGRAPH =====
class FitnessState(TypedDict):
    messages: List[HumanMessage]
    # Dados do usuário
    nome: str
    idade: int
    peso: float
    altura: float
    periodicidade: int
    objetivo: str
    experiencia: str
    # Dados processados
    imc: float
    classificacao_imc: str
    calorias_diarias: int
    # Contextos e informações
    contexto_rag: List[str]
    info_web: List[str]
    calculos_realizados: Dict[str, Any]
    # Resultado final
    treinos_gerados: List[Dict]
    total_treinos: int
    dados_validados: bool

# ===== FUNÇÕES DOS NÓS DO LANGGRAPH =====

def no_coleta_validacao(state: FitnessState) -> FitnessState:
    """Nó 1: Coleta e validação de dados do usuário"""
    msg_inicio = HumanMessage(content="🔍 Validando dados do usuário...")
    
    # Calcular IMC usando altura real do usuário
    imc = state['peso'] / (state['altura'] ** 2)
    
    # Classificação do IMC
    if imc < 18.5:
        classificacao_imc = "Abaixo do peso"
    elif imc < 25:
        classificacao_imc = "Peso normal"  
    elif imc < 30:
        classificacao_imc = "Sobrepeso"
    else:
        classificacao_imc = "Obesidade"
    
    msg_validacao = HumanMessage(content=f"✅ Dados validados: {state['nome']}, {state['idade']} anos, IMC {imc:.1f}")
    
    return {
        **state,
        "imc": round(imc, 1),
        "classificacao_imc": classificacao_imc,
        "dados_validados": True,
        "messages": state['messages'] + [msg_inicio, msg_validacao]
    }

def no_calculos_matematicos(state: FitnessState) -> FitnessState:
    """Nó 2: Cálculos matemáticos personalizados"""
    msg_calculo = HumanMessage(content="🧮 Realizando cálculos personalizados...")
    
    # TMB usando fórmula de Harris-Benedict (assumindo sexo masculino)
    tmb = 88.362 + (13.397 * state['peso']) + (4.799 * state['altura'] * 100) - (5.677 * state['idade'])
    
    # Fator de atividade baseado na periodicidade
    if state['periodicidade'] <= 3:
        fator_atividade = 1.55
    elif state['periodicidade'] <= 5:
        fator_atividade = 1.725
    else:
        fator_atividade = 1.9
    
    calorias_diarias = int(tmb * fator_atividade)
    
    # Ajustes baseados no objetivo
    if state['objetivo'] == 'emagrecimento':
        calorias_diarias = int(calorias_diarias * 0.85)  # Déficit calórico
        gasto_treino = 400
    elif state['objetivo'] == 'hipertrofia':
        calorias_diarias = int(calorias_diarias * 1.15)  # Superávit calórico
        gasto_treino = 300
    else:  # força/condicionamento
        gasto_treino = 350
    
    calculos = {
        'tmb': int(tmb),
        'fator_atividade': fator_atividade,
        'gasto_treino': gasto_treino,
        'volume_semanal': state['periodicidade'] * 45,  # minutos por semana
    }
    
    msg_resultado = HumanMessage(content=f"⚙️ TMB: {int(tmb)} kcal, Necessidade: {calorias_diarias} kcal/dia")
    
    return {
        **state,
        "calorias_diarias": calorias_diarias,
        "calculos_realizados": calculos,
        "messages": state['messages'] + [msg_calculo, msg_resultado]
    }

def no_busca_informacoes(state: FitnessState) -> FitnessState:
    """Nó 3: Busca de informações (RAG + Web usando base vetorial real)"""
    msg_busca = HumanMessage(content="🔍 Consultando base vetorial de conhecimento...")
    
    objetivo = state['objetivo']
    experiencia = state['experiencia']
    
    # ===== RAG REAL: CARREGAR E CONSULTAR BASE VETORIAL =====
    try:
        # Carregar chunks vetorizados
        with open('fitness_chunks.json', 'r', encoding='utf-8') as f:
            dados_chunks = json.load(f)
        
        chunks_disponiveis = dados_chunks['chunks']
        msg_carregamento = HumanMessage(content=f"📄 {len(chunks_disponiveis)} chunks carregados da base vetorial")
        
        # Busca semântica baseada no perfil
        contexto_rag = []
        
        # Palavras-chave para busca baseada no perfil
        keywords_busca = []
        if objetivo == 'hipertrofia':
            keywords_busca = ['hipertrofia', 'massa', 'músculo', 'crescimento', 'anabolismo']
        elif objetivo == 'emagrecimento':
            keywords_busca = ['emagrecimento', 'queima', 'gordura', 'cardio', 'metabolismo']
        elif objetivo == 'força':
            keywords_busca = ['força', 'potência', '1RM', 'neural', 'máxima']
        else:  # condicionamento
            keywords_busca = ['condicionamento', 'resistência', 'funcional', 'aeróbico']
        
        keywords_busca.extend([experiencia, 'série', 'repetição', 'treino'])
        
        # Buscar chunks relevantes
        chunks_relevantes = []
        for chunk in chunks_disponiveis:
            texto_chunk = chunk['texto'].lower()
            relevancia = 0
            for keyword in keywords_busca:
                if keyword.lower() in texto_chunk:
                    relevancia += 1
            
            if relevancia > 0:
                chunks_relevantes.append({
                    'chunk': chunk,
                    'relevancia': relevancia
                })
        
        # Ordenar por relevância e pegar os top 5
        chunks_relevantes.sort(key=lambda x: x['relevancia'], reverse=True)
        top_chunks = chunks_relevantes[:5]
        
        # Extrair contexto dos chunks mais relevantes
        for item in top_chunks:
            chunk = item['chunk']
            # Limpar e extrair informação útil
            texto_limpo = chunk['texto'].replace('###', '').replace('##', '').strip()
            if len(texto_limpo) > 50:  # Só usar chunks com conteúdo substancial
                contexto_rag.append(f"[{chunk['secao']}] {texto_limpo[:200]}...")
        
        msg_rag = HumanMessage(content=f"🧠 {len(top_chunks)} chunks relevantes encontrados para {objetivo}")
        
    except Exception as e:
        # Fallback para contexto básico se houver erro
        msg_erro = HumanMessage(content=f"⚠️ Erro ao carregar base: {e}. Usando conhecimento básico.")
        
        contexto_rag = [
            f"Protocolo {objetivo} para {experiencia}",
            f"Frequência: {state['periodicidade']}x por semana",
            "Exercícios compostos são fundamentais para todos os objetivos"
        ]
        msg_rag = msg_erro
    
    # ===== WEB SEARCH SIMULADO (COMPLEMENTAR) =====
    info_web = [
        f"Protocolo {objetivo} 2024 atualizado para {experiencia}",
        f"Periodização científica para {state['periodicidade']}x/semana",
        f"Adaptações para IMC {state['imc']} ({state['classificacao_imc']})"
    ]
    
    msg_encontrado = HumanMessage(content=f"✅ Base de conhecimento consultada com sucesso!")
    
    return {
        **state,
        "contexto_rag": contexto_rag,
        "info_web": info_web,
        "messages": state['messages'] + [msg_busca, msg_carregamento, msg_rag, msg_encontrado]
    }

def no_geracao_treinos(state: FitnessState) -> FitnessState:
    """Nó 4: Geração inteligente de treinos personalizados"""
    msg_geracao = HumanMessage(content="🏋️‍♂️ Gerando planos de treino personalizados...")
    
    objetivo = state['objetivo']
    periodicidade = state['periodicidade']
    experiencia = state['experiencia']
    imc = state['imc']
    
    # Configurações específicas por objetivo
    config_objetivo = {
        'hipertrofia': {
            'series': '3-4', 'reps': '8-12', 'descanso': '60-90s',
            'intensidade': '70-85% 1RM', 'volume_alto': True
        },
        'emagrecimento': {
            'series': '2-3', 'reps': '12-20', 'descanso': '30-45s',
            'intensidade': '60-75% 1RM', 'cardio': True
        },
        'força': {
            'series': '3-5', 'reps': '3-6', 'descanso': '2-3min',
            'intensidade': '85-95% 1RM', 'carga_alta': True
        },
        'condicionamento': {
            'series': '2-4', 'reps': '15-25', 'descanso': '30-60s',
            'intensidade': '50-70% 1RM', 'funcional': True
        }
    }
    
    config = config_objetivo.get(objetivo, config_objetivo['hipertrofia'])
    
    # Base de exercícios expandida e categorizada
    exercicios_db = {
        'peito': {
            'iniciante': ['Supino reto máquina', 'Flexão apoiada', 'Crucifixo máquina'],
            'intermediario': ['Supino reto barra', 'Supino inclinado halter', 'Crucifixo reto'],
            'avancado': ['Supino reto barra', 'Supino inclinado barra', 'Crucifixo inclinado', 'Paralelas']
        },
        'costas': {
            'iniciante': ['Puxada máquina', 'Remada máquina', 'Pullover máquina'],
            'intermediario': ['Puxada frontal', 'Remada baixa', 'Levantamento terra'],
            'avancado': ['Barra fixa', 'Remada curvada', 'Levantamento terra', 'Pulley alto']
        },
        'pernas': {
            'iniciante': ['Leg press', 'Cadeira extensora', 'Mesa flexora'],
            'intermediario': ['Agachamento guiado', 'Leg press', 'Stiff', 'Panturrilha'],
            'avancado': ['Agachamento livre', 'Agachamento frontal', 'Stiff', 'Afundo', 'Hack squat']
        },
        'ombros': {
            'iniciante': ['Desenvolvimento máquina', 'Elevação lateral máquina'],
            'intermediario': ['Desenvolvimento halter', 'Elevação lateral', 'Elevação posterior'],
            'avancado': ['Desenvolvimento militar', 'Elevação lateral', 'Elevação posterior', 'Remada alta']
        },
        'bracos': {
            'iniciante': ['Rosca direta máquina', 'Tríceps máquina'],
            'intermediario': ['Rosca direta barra', 'Tríceps testa', 'Rosca martelo'],
            'avancado': ['Rosca direta', 'Rosca alternada', 'Tríceps francês', 'Tríceps mergulho']
        }
    }
    
    # Divisão inteligente baseada na periodicidade e IMC
    if periodicidade <= 3:
        if imc > 25:  # Sobrepeso - mais cardio
            divisoes = [
                {'nome': 'Treino A - Superior', 'grupos': ['peito', 'ombros', 'bracos'], 'cardio': '15min'},
                {'nome': 'Treino B - Inferior + Core', 'grupos': ['pernas'], 'cardio': '20min'},
                {'nome': 'Treino C - Costas + Funcional', 'grupos': ['costas', 'bracos'], 'cardio': '15min'}
            ]
        else:
            divisoes = [
                {'nome': 'Treino A - Peito, Ombros, Tríceps', 'grupos': ['peito', 'ombros', 'bracos']},
                {'nome': 'Treino B - Costas, Bíceps', 'grupos': ['costas', 'bracos']},
                {'nome': 'Treino C - Pernas Completo', 'grupos': ['pernas']}
            ]
    elif periodicidade == 4:
        divisoes = [
            {'nome': 'Treino A - Peito e Tríceps', 'grupos': ['peito', 'bracos']},
            {'nome': 'Treino B - Costas e Bíceps', 'grupos': ['costas', 'bracos']},
            {'nome': 'Treino C - Pernas', 'grupos': ['pernas']},
            {'nome': 'Treino D - Ombros e Core', 'grupos': ['ombros']}
        ]
    else:  # 5-6 dias
        divisoes = [
            {'nome': 'Treino A - Peito', 'grupos': ['peito']},
            {'nome': 'Treino B - Costas', 'grupos': ['costas']},
            {'nome': 'Treino C - Pernas (Anterior)', 'grupos': ['pernas']},
            {'nome': 'Treino D - Ombros', 'grupos': ['ombros']},
            {'nome': 'Treino E - Braços', 'grupos': ['bracos']},
            {'nome': 'Treino F - Pernas (Posterior)', 'grupos': ['pernas']}
        ]
    
    # Gerar 12 treinos únicos (3 semanas de progressão)
    treinos_gerados = []
    
    for semana in range(1, 4):  # 3 semanas
        for i, divisao in enumerate(divisoes):
            if len(treinos_gerados) >= 12:
                break
                
            exercicios_treino = []
            
            for grupo in divisao['grupos']:
                exercicios_grupo = exercicios_db[grupo][experiencia]
                
                # Número de exercícios baseado na experiência e semana
                if experiencia == 'avancado':
                    num_exercicios = min(3, len(exercicios_grupo))
                elif experiencia == 'intermediario':
                    num_exercicios = min(2 + (semana - 1), len(exercicios_grupo))
                else:
                    num_exercicios = min(2, len(exercicios_grupo))
                
                # Selecionar exercícios com rotação por semana
                for j in range(num_exercicios):
                    idx = (j + (semana - 1) * 2) % len(exercicios_grupo)
                    exercicio = exercicios_grupo[idx]
                    
                    # Personalizar séries baseado na progressão semanal
                    series_base = config['series']
                    if semana == 3:  # Semana de intensificação
                        if objetivo == 'hipertrofia':
                            series_mod = '4' if '3-4' in series_base else series_base
                        else:
                            series_mod = series_base
                    else:
                        series_mod = series_base
                    
                    exercicios_treino.append({
                        'exercicio': exercicio,
                        'series': series_mod,
                        'repeticoes': config['reps'],
                        'descanso': config['descanso'],
                        'intensidade': config['intensidade'],
                        'observacao': f'Sem. {semana} - {grupo.title()}'
                    })
            
            # Adicionar cardio se necessário
            cardio_info = divisao.get('cardio', '')
            if cardio_info:
                exercicios_treino.append({
                    'exercicio': 'Cardio (esteira/bike)',
                    'series': '1',
                    'repeticoes': cardio_info,
                    'descanso': '-',
                    'intensidade': '65-75% FCmax',
                    'observacao': 'Finalizar treino'
                })
            
            treino = {
                'numero': len(treinos_gerados) + 1,
                'nome': divisao['nome'],
                'semana': semana,
                'exercicios': exercicios_treino,
                'duracao_estimada': 60 + len(exercicios_treino) * 3
            }
            
            treinos_gerados.append(treino)
    
    msg_concluido = HumanMessage(content=f"✅ {len(treinos_gerados)} treinos personalizados gerados com sucesso!")
    
    return {
        **state,
        "treinos_gerados": treinos_gerados,
        "total_treinos": len(treinos_gerados),
        "messages": state['messages'] + [msg_geracao, msg_concluido]
    }

# ===== CONSTRUÇÃO DO GRAFO LANGGRAPH =====
def criar_grafo_fitness():
    """Cria e compila o grafo LangGraph para fitness"""
    # Criar builder
    builder = StateGraph(FitnessState)
    
    # Adicionar nós
    builder.add_node("validacao", no_coleta_validacao)
    builder.add_node("calculos", no_calculos_matematicos) 
    builder.add_node("busca_info", no_busca_informacoes)
    builder.add_node("gera_treinos", no_geracao_treinos)
    
    # Definir fluxo
    builder.add_edge(START, "validacao")
    builder.add_edge("validacao", "calculos")
    builder.add_edge("calculos", "busca_info")
    builder.add_edge("busca_info", "gera_treinos")
    builder.add_edge("gera_treinos", END)
    
    # Compilar com memória
    memory = MemorySaver()
    return builder.compile(checkpointer=memory)

# Criar grafo global
fitness_graph = criar_grafo_fitness()

print("✅ LangGraph personalizado criado!")
print("🔄 Fluxo: validacao → calculos → busca_info → gera_treinos")
print("🎯 Sistema agora usa RAG REAL com base vetorial!")
print("📊 Base de chunks será consultada dinamicamente por objetivo/experiência")

✅ LangGraph personalizado criado!
🔄 Fluxo: validacao → calculos → busca_info → gera_treinos
🎯 Sistema agora usa RAG REAL com base vetorial!
📊 Base de chunks será consultada dinamicamente por objetivo/experiência


## 4️⃣ Entrada de Dados do Usuário

Preencha seus dados abaixo para gerar seu plano personalizado:

In [ ]:
# 📝 ENTRADA DE DADOS DO USUÁRIO
# Modifique os valores abaixo com seus dados pessoais:

# ========================================
# 👤 SEUS DADOS AQUI - MODIFIQUE CONFORME NECESSÁRIO
# ========================================

# Seu nome
nome = "Lionel" 

# Sua idade em anos         
idade = 38  
     
# Seu peso em kg                 
peso = 68    

# Sua altura em metros (ex: 1.75)                  
altura = 1.70

# Quantos dias por semana você treina (2-6)               
periodicidade = 4

# Opções: "hipertrofia", "emagrecimento", "força", "condicionamento"               
objetivo = "condicionamento"    

# Opções: "iniciante", "intermediario", "avancado"           
experiencia = "intermediario"        

# ========================================
# 🔄 PROCESSAMENTO AUTOMÁTICO DOS DADOS
# ========================================

# Criar o estado inicial com seus dados
estado = {
	'messages': [],
	'nome': nome,
	'idade': idade,
	'peso': peso,
	'altura': altura,
	'periodicidade': periodicidade,
	'objetivo': objetivo.lower().strip(),
	'experiencia': experiencia.lower().strip(),
	# Campos que serão preenchidos pelo pipeline
	'imc': 0.0,
	'classificacao_imc': '',
	'calorias_diarias': 0,
	'contexto_rag': [],
	'info_web': [],
	'calculos_realizados': {},
	'treinos_gerados': [],
	'total_treinos': 0,
	'dados_validados': False
}

# Mostrar resumo dos dados inseridos
print("✅ DADOS CONFIGURADOS:")
print(f"👤 Nome: {nome}")
print(f"🎂 Idade: {idade} anos")
print(f"⚖️ Peso: {peso} kg")
print(f"📏 Altura: {altura} m")
print(f"📅 Periodicidade: {periodicidade}x por semana")
print(f"🎯 Objetivo: {objetivo}")
print(f"💪 Experiência: {experiencia}")

# Calcular IMC automaticamente
imc = peso / (altura ** 2)
print(f"📊 IMC calculado: {imc:.1f}")

print(f"\n✅ Estado inicial criado com sucesso!")
print(f"🔄 Execute a próxima célula para processar o plano de treino.")

✅ DADOS CONFIGURADOS:
👤 Nome: Mariana Gomes Souza
🎂 Idade: 27 anos
⚖️ Peso: 72 kg
📏 Altura: 1.63 m
📅 Periodicidade: 2x por semana
🎯 Objetivo: condicionamento
💪 Experiência: iniciante
📊 IMC calculado: 27.1

✅ Estado inicial criado com sucesso!
🔄 Execute a próxima célula para processar o plano de treino.


## 5️⃣ Execução do Pipeline LangGraph

Aqui o pipeline é executado, passando pelos nós definidos, e o plano é gerado.

In [63]:
# 🚀 EXECUÇÃO DO PIPELINE LANGGRAPH PERSONALIZADO

print("🏋️‍♂️ Iniciando pipeline LangGraph personalizado...")
print(f"👤 Processando para: {nome}")

# Criar estado inicial com TODOS os dados do usuário
estado_inicial = {
    'messages': [],
    'nome': nome,
    'idade': idade,
    'peso': peso,
    'altura': altura,
    'periodicidade': periodicidade,
    'objetivo': objetivo,
    'experiencia': experiencia,
    # Campos que serão preenchidos pelo pipeline
    'imc': 0.0,
    'classificacao_imc': '',
    'calorias_diarias': 0,
    'contexto_rag': [],
    'info_web': [],
    'calculos_realizados': {},
    'treinos_gerados': [],
    'total_treinos': 0,
    'dados_validados': False
}

try:
    # Executar o grafo LangGraph
    config = {"configurable": {"thread_id": f"fitness_session_{nome}_{periodicidade}_{objetivo}"}}
    
    print("🔄 Executando nós do LangGraph...")
    resultado = fitness_graph.invoke(estado_inicial, config=config)
    
    print("\n" + "="*60)
    print("🎉 PIPELINE LANGGRAPH CONCLUÍDO!")
    print("="*60)
    
    # Mostrar histórico do processamento
    print("\n📝 Histórico do processamento:")
    for i, msg in enumerate(resultado['messages'], 1):
        print(f"  {i}. {msg.content}")
    
    print(f"\n📊 RESULTADOS PERSONALIZADOS:")
    print(f"⚖️ IMC: {resultado['imc']} ({resultado['classificacao_imc']})")
    print(f"🔥 Calorias diárias: {resultado['calorias_diarias']:,} kcal")
    print(f"🏋️‍♂️ Treinos gerados: {resultado['total_treinos']}")
    print(f"🎯 Especialização: {resultado['objetivo']} para {resultado['experiencia']}")
    
    # Atualizar estado global para geração do PDF
    estado = resultado
    erro = False
    
    print(f"\n✅ Sistema personalizado executado com sucesso!")
    
except Exception as e:
    print(f"❌ Erro no pipeline LangGraph: {e}")
    import traceback
    traceback.print_exc()
    erro = True

🏋️‍♂️ Iniciando pipeline LangGraph personalizado...
👤 Processando para: Mariana Gomes Souza
🔄 Executando nós do LangGraph...

🎉 PIPELINE LANGGRAPH CONCLUÍDO!

📝 Histórico do processamento:
  1. 🔍 Validando dados do usuário...
  2. ✅ Dados validados: Mariana Gomes Souza, 27 anos, IMC 27.1
  3. 🧮 Realizando cálculos personalizados...
  4. ⚙️ TMB: 1681 kcal, Necessidade: 2606 kcal/dia
  5. 🔍 Consultando base vetorial de conhecimento...
  6. 📄 22 chunks carregados da base vetorial
  7. 🧠 5 chunks relevantes encontrados para condicionamento
  8. ✅ Base de conhecimento consultada com sucesso!
  9. 🏋️‍♂️ Gerando planos de treino personalizados...
  10. ✅ 9 treinos personalizados gerados com sucesso!

📊 RESULTADOS PERSONALIZADOS:
⚖️ IMC: 27.1 (Sobrepeso)
🔥 Calorias diárias: 2,606 kcal
🏋️‍♂️ Treinos gerados: 9
🎯 Especialização: condicionamento para iniciante

✅ Sistema personalizado executado com sucesso!

🎉 PIPELINE LANGGRAPH CONCLUÍDO!

📝 Histórico do processamento:
  1. 🔍 Validando dados do 

## 6️⃣ Geração do Relatório em PDF

Clique para gerar e baixar seu plano de treino em PDF.

In [64]:
# 🏋️‍♂️ GERAÇÃO DE RELATÓRIO PDF PERSONALIZADO COM LANGGRAPH
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch, cm
from reportlab.lib import colors
from datetime import datetime
import os
import re

def gerar_pdf_langgraph(estado_langgraph):
    """Gera PDF usando dados processados pelo LangGraph"""
    
    # Nome do arquivo personalizado
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nome_arquivo = f"Plano_Treino_{estado_langgraph['nome'].replace(' ', '_')}_{estado_langgraph['objetivo']}_{timestamp}.pdf"
    
    # Configurar documento
    doc = SimpleDocTemplate(nome_arquivo, pagesize=A4, 
                          rightMargin=2*cm, leftMargin=2*cm,
                          topMargin=2*cm, bottomMargin=2*cm)
    
    styles = getSampleStyleSheet()
    story = []
    
    # Estilos personalizados
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Title'],
        fontSize=24,
        spaceAfter=30,
        textColor=colors.darkblue,
        alignment=1  # Centralizado
    )
    
    subtitle_style = ParagraphStyle(
        'CustomSubtitle',
        parent=styles['Heading2'],
        fontSize=14,
        spaceAfter=15,
        textColor=colors.darkgreen,
        alignment=1
    )
    
    heading_style = ParagraphStyle(
        'CustomHeading',
        parent=styles['Heading2'],
        fontSize=16,
        spaceBefore=20,
        spaceAfter=10,
        textColor=colors.darkred
    )
    
    # === CAPA PERSONALIZADA ===
    story.append(Spacer(1, 2*cm))
    story.append(Paragraph("🏋️‍♂️ PLANO DE TREINO PERSONALIZADO", title_style))
    story.append(Paragraph("Powered by AI LangGraph", subtitle_style))
    story.append(Spacer(1, 1*cm))
    
    story.append(Paragraph(f"CLIENTE: {estado_langgraph['nome'].upper()}", subtitle_style))
    story.append(Paragraph(f"OBJETIVO: {estado_langgraph['objetivo'].upper()}", subtitle_style))
    story.append(Paragraph(f"Elaborado em: {datetime.now().strftime('%d/%m/%Y às %H:%M')}", subtitle_style))
    story.append(Spacer(1, 2*cm))
    
    # Dados calculados pelo LangGraph
    dados_capa = [
        ["PERFIL PERSONALIZADO", "VALORES CALCULADOS"],
        ["Nome Completo", estado_langgraph['nome']],
        ["Idade", f"{estado_langgraph['idade']} anos"],
        ["Peso Atual", f"{estado_langgraph['peso']} kg"],
        ["Altura", f"{estado_langgraph['altura']} m"],
        ["IMC Calculado", f"{estado_langgraph['imc']} ({estado_langgraph['classificacao_imc']})"],
        ["Objetivo Principal", estado_langgraph['objetivo'].title()],
        ["Nível de Experiência", estado_langgraph['experiencia'].title()],
        ["Frequência Semanal", f"{estado_langgraph['periodicidade']} dias"],
        ["Calorias Diárias", f"{estado_langgraph['calorias_diarias']:,} kcal"],
        ["TMB Calculada", f"{estado_langgraph['calculos_realizados'].get('tmb', 'N/A'):,} kcal"],
        ["Gasto por Treino", f"{estado_langgraph['calculos_realizados'].get('gasto_treino', 'N/A')} kcal"],
        ["Volume Semanal", f"{estado_langgraph['calculos_realizados'].get('volume_semanal', 'N/A')} min"]
    ]
    
    tabela_capa = Table(dados_capa, colWidths=[7*cm, 7*cm])
    tabela_capa.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.darkblue),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 12),
        ('FONTSIZE', (0, 1), (-1, -1), 10),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.lightgrey),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.lightgrey])
    ]))
    
    story.append(tabela_capa)
    story.append(PageBreak())
    
    # === HISTÓRICO DO PROCESSAMENTO LANGGRAPH ===
    story.append(Paragraph("🤖 PROCESSAMENTO IA - LANGGRAPH", heading_style))
    story.append(Paragraph("Este plano foi gerado através de um pipeline inteligente que processou seus dados através dos seguintes estágios:", styles['Normal']))
    story.append(Spacer(1, 0.5*cm))
    
    for i, msg in enumerate(estado_langgraph['messages'], 1):
        story.append(Paragraph(f"<b>{i}.</b> {msg.content}", styles['Normal']))
    
    story.append(Spacer(1, 1*cm))
    
    # === FUNÇÃO PARA PROCESSAR INFORMAÇÕES RAG ===
    def processar_info_rag(texto_raw):
        """Processa texto RAG em formato estruturado para tabela"""
        # Extrair categoria do texto [CATEGORIA]
        categoria_match = re.search(r'\[([^\]]+)\]', texto_raw)
        categoria = categoria_match.group(1).title() if categoria_match else "Geral"
        
        # Remover a parte da categoria do texto
        texto_limpo = re.sub(r'\[[^\]]+\]', '', texto_raw).strip()
        
        # Extrair informações estruturadas
        informacoes = []
        
        # Buscar padrões de informação estruturada
        padroes = [
            (r'Volume[:\s]*([^-]+)', 'Volume'),
            (r'Intensidade[:\s]*([^-]+)', 'Intensidade'),  
            (r'Frequência[:\s]*([^-]+)', 'Frequência'),
            (r'Descanso[:\s]*([^-]+)', 'Descanso'),
            (r'Intervalo[:\s]*([^-]+)', 'Intervalo'),
            (r'Modalidades[:\s]*([^-]+)', 'Modalidades'),
            (r'Músculos[:\s]*([^-]+)', 'Músculos'),
            (r'Execução[:\s]*([^-]+)', 'Execução'),
            (r'Dicas[:\s]*([^-]+)', 'Dicas')
        ]
        
        for padrao, nome in padroes:
            match = re.search(padrao, texto_limpo, re.IGNORECASE)
            if match:
                valor = match.group(1).strip()
                # Limpar valor removendo ** e outros marcadores
                valor = re.sub(r'\*\*([^*]+)\*\*', r'\1', valor)
                valor = valor.replace('**', '').strip()
                if len(valor) > 5:  # Só adicionar se tiver conteúdo substancial
                    informacoes.append([nome, valor[:100] + "..." if len(valor) > 100 else valor])
        
        return categoria, informacoes
    
    # === CONTEXTO RAG PROFISSIONAL ===
    if estado_langgraph.get('contexto_rag'):
        story.append(Paragraph("📚 BASE DE CONHECIMENTO CONSULTADA", heading_style))
        story.append(Paragraph("Informações especializadas extraídas da base vetorial de conhecimento fitness:", styles['Normal']))
        story.append(Spacer(1, 0.5*cm))
        
        for i, info_raw in enumerate(estado_langgraph['contexto_rag'], 1):
            categoria, informacoes = processar_info_rag(info_raw)
            
            if informacoes:  # Só criar tabela se tiver informações estruturadas
                # Cabeçalho da categoria
                story.append(Paragraph(f"<b>#{i}. {categoria.upper()}</b>", 
                                     ParagraphStyle('CategoriaRAG', parent=styles['Heading3'],
                                                  fontSize=12, textColor=colors.darkblue, 
                                                  spaceBefore=15, spaceAfter=5)))
                
                # Criar tabela com as informações
                dados_info = [["PARÂMETRO", "ESPECIFICAÇÃO"]]
                dados_info.extend(informacoes)
                
                tabela_info = Table(dados_info, colWidths=[4*cm, 10*cm])
                tabela_info.setStyle(TableStyle([
                    ('BACKGROUND', (0, 0), (-1, 0), colors.lightblue),
                    ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
                    ('ALIGN', (0, 0), (0, -1), 'LEFT'),
                    ('ALIGN', (1, 0), (1, -1), 'LEFT'),
                    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                    ('FONTSIZE', (0, 0), (-1, 0), 10),
                    ('FONTSIZE', (0, 1), (-1, -1), 9),
                    ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
                    ('TOPPADDING', (0, 1), (-1, -1), 4),
                    ('BOTTOMPADDING', (0, 1), (-1, -1), 4),
                    ('BACKGROUND', (0, 1), (-1, -1), colors.white),
                    ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
                    ('VALIGN', (0, 0), (-1, -1), 'TOP'),
                    ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.lightgrey])
                ]))
                
                story.append(tabela_info)
                story.append(Spacer(1, 0.3*cm))
            else:
                # Fallback para texto simples se não conseguir estruturar
                texto_simples = re.sub(r'\[[^\]]+\]', '', info_raw)[:200] + "..."
                story.append(Paragraph(f"<b>#{i}.</b> {texto_simples}", styles['Normal']))
                story.append(Spacer(1, 0.2*cm))
        
        story.append(Spacer(1, 0.5*cm))
    
    # === FONTES COMPLEMENTARES MELHORADO ===
    if estado_langgraph.get('info_web'):
        story.append(Paragraph("🌐 FONTES COMPLEMENTARES", heading_style))
        
        # Criar tabela para fontes web também
        dados_web = [["#", "FONTE CONSULTADA", "RELEVÂNCIA"]]
        for i, info in enumerate(estado_langgraph['info_web'], 1):
            relevancia = "Alta" if estado_langgraph['objetivo'].lower() in info.lower() else "Média"
            dados_web.append([str(i), info, relevancia])
        
        tabela_web = Table(dados_web, colWidths=[1*cm, 10*cm, 3*cm])
        tabela_web.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.lightgreen),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
            ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 10),
            ('FONTSIZE', (0, 1), (-1, -1), 9),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
            ('BACKGROUND', (0, 1), (-1, -1), colors.white),
            ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.lightgrey])
        ]))
        
        story.append(tabela_web)
    
    story.append(PageBreak())
    
    # === TREINOS PERSONALIZADOS GERADOS PELO LANGGRAPH ===
    story.append(Paragraph("🏋️‍♂️ PROGRAMA DE TREINOS PERSONALIZADO", heading_style))
    
    story.append(Paragraph(f"Total de <b>{estado_langgraph['total_treinos']} treinos únicos</b> gerados especificamente para seu perfil:", styles['Normal']))
    story.append(Spacer(1, 0.5*cm))
    
    # Usar os treinos gerados pelo LangGraph
    treinos_langgraph = estado_langgraph.get('treinos_gerados', [])
    
    if not treinos_langgraph:
        story.append(Paragraph("❌ Nenhum treino foi gerado pelo LangGraph.", styles['Normal']))
    else:
        for treino in treinos_langgraph:
            # Cabeçalho do treino
            story.append(Paragraph(f"TREINO {treino['numero']} - {treino['nome']}", 
                                 ParagraphStyle('TreinoTitle', parent=styles['Heading3'], 
                                              fontSize=14, textColor=colors.darkblue, 
                                              spaceBefore=20, spaceAfter=10)))
            
            story.append(Paragraph(f"Semana {treino['semana']} | Duração estimada: {treino.get('duracao_estimada', 75)} min", 
                                 styles['Normal']))
            
            # Tabela de exercícios do LangGraph
            dados_exercicios = [
                ["EXERCÍCIO", "SÉRIES", "REPETIÇÕES", "DESCANSO", "OBSERVAÇÕES"]
            ]
            
            for ex in treino['exercicios']:
                dados_exercicios.append([
                    ex['exercicio'],
                    ex['series'],
                    ex['repeticoes'], 
                    ex['descanso'],
                    ex.get('observacao', ex.get('intensidade', ''))
                ])
            
            tabela_treino = Table(dados_exercicios, 
                                colWidths=[5*cm, 1.5*cm, 2*cm, 1.8*cm, 3.7*cm])
            
            tabela_treino.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.lightblue),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
                ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, 0), 9),
                ('FONTSIZE', (0, 1), (-1, -1), 8),
                ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
                ('BACKGROUND', (0, 1), (-1, -1), colors.white),
                ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
                ('VALIGN', (0, 0), (-1, -1), 'MIDDLE')
            ]))
            
            story.append(tabela_treino)
            story.append(Spacer(1, 0.3*cm))
            
            # Nova página a cada 2 treinos para melhor organização
            if treino['numero'] % 2 == 0 and treino['numero'] < len(treinos_langgraph):
                story.append(PageBreak())
    
    # === PROGRESSÃO PERSONALIZADA ===
    story.append(PageBreak())
    story.append(Paragraph("📈 PROGRESSÃO PERSONALIZADA", heading_style))
    
    # Progressão baseada no objetivo específico
    if estado_langgraph['objetivo'] == 'hipertrofia':
        progressao_texto = """
        <b>Protocolo específico para HIPERTROFIA:</b><br/>
        • Semana 1-2: Adaptação neural e técnica (cargas moderadas)<br/>
        • Semana 3-4: Aumento de volume (séries extras nos exercícios principais)<br/>
        • Semana 5-6: Intensificação (aumento de 5-10% nas cargas)<br/>
        • Semana 7: Deload (redução de 20% volume e intensidade)<br/>
        • Semana 8+: Novo macrociclo com cargas base aumentadas<br/>
        """
    elif estado_langgraph['objetivo'] == 'emagrecimento':
        progressao_texto = """
        <b>Protocolo específico para EMAGRECIMENTO:</b><br/>
        • Semana 1-2: Estabelecer rotina e condicionamento base<br/>
        • Semana 3-4: Introduzir supersets e reduzir descansos<br/>
        • Semana 5-6: Adicionar circuitos e exercícios compostos<br/>
        • Semana 7: Manutenção com foco na técnica<br/>
        • Semana 8+: Progressão em densidade e complexidade<br/>
        """
    elif estado_langgraph['objetivo'] == 'força':
        progressao_texto = """
        <b>Protocolo específico para FORÇA:</b><br/>
        • Semana 1-2: Estabelecer 1RM e técnica nos básicos<br/>
        • Semana 3-4: Trabalhar em 85-90% com foco neural<br/>
        • Semana 5-6: Picos de intensidade (90-95% 1RM)<br/>
        • Semana 7: Deload obrigatório (50-60% das cargas)<br/>
        • Semana 8+: Novo ciclo com 1RM aumentado<br/>
        """
    else:
        progressao_texto = """
        <b>Protocolo específico para CONDICIONAMENTO:</b><br/>
        • Semana 1-2: Base aeróbia e adaptação muscular<br/>
        • Semana 3-4: Introduzir intervalos de alta intensidade<br/>
        • Semana 5-6: Trabalho anaeróbio e potência<br/>
        • Semana 7: Recuperação ativa e técnica<br/>
        • Semana 8+: Progressão em complexidade e intensidade<br/>
        """
    
    story.append(Paragraph(progressao_texto, styles['Normal']))
    
    # === ASSINATURA PERSONALIZADA ===
    story.append(Spacer(1, 2*cm))
    story.append(Paragraph("_" * 60, styles['Normal']))
    story.append(Paragraph("AI FITNESS COACH - Personal Trainer Digital", 
                         ParagraphStyle('Signature', parent=styles['Normal'],
                                      alignment=1, fontSize=12, 
                                      textColor=colors.darkblue)))
    
    story.append(Paragraph(f"Plano gerado via LangGraph | Cliente: {estado_langgraph['nome']} | {datetime.now().strftime('%d/%m/%Y às %H:%M')}", 
                         ParagraphStyle('CREF', parent=styles['Normal'],
                                      alignment=1, fontSize=9, 
                                      textColor=colors.grey)))
    
    # Construir PDF
    doc.build(story)
    
    print(f"\n🎉 RELATÓRIO PERSONALIZADO GERADO!")
    print(f"📄 Arquivo: {nome_arquivo}")
    print(f"📊 {len(treinos_langgraph)} treinos únicos do LangGraph")
    print(f"🎯 Especializado para: {estado_langgraph['objetivo']} ({estado_langgraph['experiencia']})")
    print(f"👤 Cliente: {estado_langgraph['nome']}")
    print(f"🤖 Processado por: {len(estado_langgraph['messages'])} etapas de IA")
    
    return nome_arquivo

# ===== EXECUÇÃO DA GERAÇÃO DO PDF =====
if not erro and 'estado' in globals() and estado.get('treinos_gerados'):
    print("\n🚀 Gerando PDF com dados do LangGraph...")
    nome_arquivo_final = gerar_pdf_langgraph(estado)
    print(f"\n✅ PDF personalizado salvo como: {nome_arquivo_final}")
elif erro:
    print("\n❌ Corrija os erros antes de gerar o PDF")
elif 'estado' not in globals():
    print("\n⚠️ Execute a célula anterior (pipeline) primeiro!")
else:
    print("\n❌ Nenhum treino foi gerado pelo LangGraph. Verifique a execução do pipeline.")


🚀 Gerando PDF com dados do LangGraph...

🎉 RELATÓRIO PERSONALIZADO GERADO!
📄 Arquivo: Plano_Treino_Mariana_Gomes_Souza_condicionamento_20251005_203333.pdf
📊 9 treinos únicos do LangGraph
🎯 Especializado para: condicionamento (iniciante)
👤 Cliente: Mariana Gomes Souza
🤖 Processado por: 10 etapas de IA

✅ PDF personalizado salvo como: Plano_Treino_Mariana_Gomes_Souza_condicionamento_20251005_203333.pdf

🎉 RELATÓRIO PERSONALIZADO GERADO!
📄 Arquivo: Plano_Treino_Mariana_Gomes_Souza_condicionamento_20251005_203333.pdf
📊 9 treinos únicos do LangGraph
🎯 Especializado para: condicionamento (iniciante)
👤 Cliente: Mariana Gomes Souza
🤖 Processado por: 10 etapas de IA

✅ PDF personalizado salvo como: Plano_Treino_Mariana_Gomes_Souza_condicionamento_20251005_203333.pdf
